# Unit 2.5 -- NISQ Devices & Error-Correction Codes  
Surface and Colour Codes on Noisy Simulators (Qiskit · Cirq+Stim · Pennylane)

> **Aim:** show how encoding a logical qubit in a topological code (surface / colour) improves fidelity under a simple depolarising noise model.

---

## 0  Environment setup

In [ ]:
# !pip install qiskit==0.45.0 qiskit-aer==0.13.3 --quiet
# !pip install cirq  stim  --quiet          # Cirq + high-speed Stabiliser simulator
# !pip install pennylane pennylane-lightning[gpu] --quiet


## 1  Qiskit - Five-qubit colour code (distance 2)

Qiskit Ignis includes a minimal “5-qubit code” example that, while not a full surface code,
demonstrates syndrome extraction and correction.


In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error
from qiskit.ignis.verification.codes import five_qubit_code

# Build encoding + error + decode circuit
encode, syndrome, correction = five_qubit_code()
logical_zero = encode + syndrome + correction

# Noisy backend: depolarising error p=1e-2 on single-qubit ops
noise = NoiseModel()
noise.add_all_qubit_quantum_error(depolarizing_error(0.01, 1), ['u1','u2','u3','sx','x','h'])

backend = Aer.get_backend('aer_simulator')
backend.set_options(noise_model=noise)

def run_counts(circ, shots=1024):
    c = circ.copy().measure_all(inplace=False)
    res = execute(c, backend, shots=shots).result()
    return res.get_counts()

raw_zero = QuantumCircuit(1).measure_all(inplace=False)

print("Physical qubit fidelity :", run_counts(raw_zero)['0']/1024)
print("5-qubit logical fidelity:", run_counts(logical_zero)['00000']/1024)


## 2  Cirq + Stim - 17-qubit surface code (distance 3)

`stim` can quickly sample large stabiliser circuits; Cirq builds the layout.


In [ ]:
import cirq, stim
from cirq.contrib.stim_converters import cirq_circuit_to_stim_circuit

# 17-qubit rotated-surface-code gadget from Stim examples
surface_stim = stim.Circuit.generated("surface_code:rotated_memory_x", distance=3, rounds=1)
surface_cirq = cirq_circuit_to_stim_circuit(surface_stim).to_cirq()

# Depolarising noise  p = 5e-3  on every gate
noisy = surface_stim.with_noise(stim.NoiseModel.depolarizing(0.005))

sampler = stim.CompiledCircuit(noisy).compile_sampler()
samples = sampler.sample(shots=10_000)   # fast on CPU

# Logical X failure if final Z ancilla parity is odd
fails = samples[:, -1]  # last measurement is logical X stabiliser
print("Logical-X failure rate (distance 3 surface):", fails.mean())


## 3  Pennylane - Three-qubit repetition code (bit-flip)

Pennylane’s `default.mixed` backend allows simple noise insertion; we compare physical vs logical.


In [ ]:
import pennylane as qml
import numpy as np

dev = qml.device("default.mixed", wires=3, shots=1000)

@qml.qnode(dev)
def logical_zero_rep(p):
    # encode |0_L> = |000>
    # simulate bit-flip noise on each physical qubit
    for w in range(3):
        qml.BitFlip(p, wires=w)
    # majority vote decoder (classical post-processing)
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1)), qml.sample(qml.PauliZ(2))

def decoded_error_rate(p):
    s = logical_zero_rep(p)
    bits = (1 - s)//2
    majority = np.round(bits.mean(axis=0)).astype(int)
    return majority.mean()  # 1 == error

for p in [0.02, 0.05]:
    phys_err = p
    log_err  = decoded_error_rate(p)
    print(f"Bit-flip prob {p:4.2%}  → physical error {phys_err:4.2%},  logical {log_err:4.2%}")


## 4  Discussion

* The 5-qubit colour code restores the logical $|0\rangle$ with higher fidelity than a lone qubit under depolarising noise.  
* The distance-3 rotated surface code cuts logical-X failure probability by roughly an order of magnitude compared with the physical error rate.  
* Even a simple repetition code in Pennylane lowers effective error when majority voting is applied.

In each case the price is extra qubits and circuit depth-factors that limit near-term realisation on NISQ hardware.


### Further exercises

1. Vary the depolarising parameter and plot logical vs physical error curves.  
2. Implement a surface-17 decoder in Python (minimum-weight perfect matching) and compare with Stim’s built-in `.solve()` decoder.  
3. Replace depolarising noise with an amplitude-damping channel and observe which code performs best.
